In [4]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
import requests
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo

In [5]:
def init_browser(): 
# Use splinter to start up a Chrome driver
    executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
    browser = Browser('chrome', **executable_path, headless=False)
init_browser()

In [7]:
def scrape_nasa_news():
    news = {}
    url =  'https://mars.nasa.gov/news'
    response = requests.get(url)
    soup = bs(response.text, 'html.parser')
    #result = soup.find('li', class_='slide')
    result = soup.find('div', class_='slide')
    # Extract the most recent news' title and content
    #news_title = result.find('li', class_="content_title").find('a').text
    news_title = result.find('div', class_="content_title").find('a').text.strip()
    #news_p = result.find('li', class_="article_teaser_body").text
    news_p = result.a.text.strip()
    news["news_title"] = news_title
    news["news_p"] = news_p
    return news
scrape_nasa_news()

{'news_title': 'NASA Prepares for Moon and Mars With New Addition to Its Deep Space Network',
 'news_p': 'Robotic spacecraft will be able to communicate with the dish using radio waves and lasers.'}

In [8]:
def scrape_featured_image():
    url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    base_url ="https://www.jpl.nasa.gov"
    
    response = requests.get(url)
    soup = bs(response.text, 'lxml')
    article = soup.find("article")
    
    featured_img_url = base_url + article.a['data-fancybox-href']
    return featured_img_url
scrape_featured_image()

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA16919_ip.jpg'

In [9]:
def scrape_latest_twitt():
    url = "https://twitter.com/marswxreport?lang=en"
    response = requests.get(url)
    soup = bs(response.text, 'lxml')
    
    mars_weather = soup.find_all("p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text")[0].text
    return mars_weather
scrape_latest_twitt()

'InSight sol 442 (2020-02-23) low -94.1ºC (-137.3ºF) high -10.5ºC (13.0ºF)\nwinds from the SSE at 6.2 m/s (13.8 mph) gusting to 21.1 m/s (47.3 mph)\npressure at 6.30 hPapic.twitter.com/lfdFlvxVxe'

In [10]:
def scrape_mars_facts():
    url = 'http://space-facts.com/mars/'
    # Use Panda's `read_html` to parse the url
    tables = pd.read_html(url)
    df = tables[0]
    df.columns = ['description', 'value']
    df.set_index('description', inplace=True)
    return df.to_html()
scrape_mars_facts()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

In [11]:
def scrape_hemispheres():
    url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
    response = requests.get(url)
    soup = bs(response.text, 'lxml')
    
    hemispheres = []
    results = soup.find_all("div", class_="description")
    
    for result in results:
        hemispheres.append(result.h3.text)
        
    # hemispheres = ['Cerberus', 'Schiaparelli', 'Syrtis Major', 'Valles Marineris']
    return hemispheres
scrape_hemispheres()

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [12]:
def scrape_hemisphere_info():
    hemispheres = scrape_hemispheres()
    hemisphere_info = []

In [15]:
def scrape_hemisphere_info():
    hemispheres = scrape_hemispheres()
    hemisphere_info = []
    
    
    browser = init_browser()
    
    url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
    browser.visit(url)
    
    for hemi in hemispheres:
        #  Hemisphere
        browser.click_link_by_partial_text(f'{hemi}')
        
        # create HTML object, Parse HTML with Beautiful Soup
        html = browser.html
        soup = bs(html, 'html.parser')
        # Initialize the dict
        hemi_dict = {}
        # Scrape the incomplete URL
        inc_url = soup.find('img', class_='wide-image')['src']
        # Construct the complete URL
        img_url = 'https://astrogeology.usgs.gov' + inc_url
        # Store the data in a dictionary then add that dictionary to the tracking list
        hemi_dict["title"] = f'{hemi}'
        hemi_dict["img_url"] = img_url
        hemisphere_info.append(hemi_dict)
        
        # Return to the original page
        browser.click_link_by_partial_text('Back')
        
    browser.quit()
    return hemisphere_info

scrape_hemisphere_info()

AttributeError: 'NoneType' object has no attribute 'visit'

In [ ]:
def scrape():
    mars_facts = {}
    mars_facts["news_title"] = scrape_nasa_news()["news_title"]
    mars_facts["news_p"] = scrape_nasa_news()["news_p"]
    mars_facts["featured_image_url"] = scrape_featured_image()
    mars_facts["mars_weather"] = scrape_latest_twitt()
    mars_facts["mars_facts_table"] = scrape_mars_facts()
    mars_facts["hemisphere_info"] = scrape_hemisphere_info()
    # Return the dictionary
    return mars_facts
scrape()